### Import drive, dataset, helper functions

In [1]:
from pathlib import Path
import os
import sys

sys.path.append(str(Path(os.getcwd()).parents[1]))
from Helper_Functions import common_utils
directory_to_extract_to = os.getcwd()

common_utils.load_data_from_one_drive(directory_to_extract_to, "recommendation_paths", "apriori_store")

Extracting : 100%|██████████| 1/1 [00:01<00:00,  1.92s/it]


### Import libraries

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import networkx as nx
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori,association_rules
import matplotlib.pyplot as plt
plt.style.use('default')

### Read dataset

In [ ]:
# read the dataset in from the excel file
df = pd.read_excel('Online Retail.xlsx')

In [ ]:
df1 = df

In [ ]:
df1.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [ ]:
df1.shape

(541909, 8)

### Data cleaning

Steps in data cleaning:
* Convert stockcode to string because we have some values in alphanumeric and some in numbers
* Convert this string into uppercase for data uniformity

In [ ]:
# Prepend '_' to StockCode
df1['StockCode'] = df1['StockCode'].apply(lambda x: '_'+str(x))

In [ ]:
df1['StockCode'] = df1['StockCode'].str.upper()

In [ ]:
df1

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,_85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,_71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,_84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,_84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,_84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,_22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,_22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,_23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,_23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


In [ ]:
df2 = df1
df3 = df1

In [ ]:
# Dropping dupicate rows
df2 = df2[~df2.duplicated()]

In [ ]:
df2.shape

(536641, 8)

In [ ]:
# In dataset there are some damaged products, those have their descriptions in lower case, 
# We only want those descriptions that are completely upper case
df2 = df2[
    df2['Description'].str.upper() == df2['Description']
]

In [ ]:
df2.shape

(532160, 8)

In [ ]:
df2 = df2.reset_index(drop=True)

In [ ]:
df2

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,_85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,_71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,_84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,_84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,_84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
532155,581587,_22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
532156,581587,_22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
532157,581587,_23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
532158,581587,_23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


### Data Visualisation

To get a visualisation of top most frequently bought items

In [ ]:
# Gather All Items of Each Transactions into Numpy Array
transaction = []
for i in range(0, df2.shape[0]):
    transaction.append(df2['StockCode'][i])

transaction = np.array(transaction)

In [ ]:
transaction

array(['_85123A', '_71053', '_84406B', ..., '_23254', '_23255', '_22138'],
      dtype='<U10')

In [ ]:
# Transform these into a Pandas DataFrame, with  their frequency as incident count
df_trans = pd.DataFrame(transaction, columns=["items"]) 
df_trans["incident_count"] = 1 # Put 1 to Each Item For Making Countable Table, to be able to perform Group By

# Making a new appropriate pandas dataFrame for visualizations  
df_table = df_trans.groupby("items").sum().sort_values("incident_count", ascending=False).reset_index()

# Top 10 most frequent
df_table.head(10).style.background_gradient(cmap='Blues')

,items,incident_count
0,_85123A,2367
1,_22423,2189
2,_85099B,2156
3,_47566,1720
4,_20725,1626
5,_84879,1488
6,_22197,1468
7,_22720,1465
8,_21212,1367
9,_22383,1328


In [ ]:
# Top 30 most frequent items

df_table["all"] = "all"

fig = px.treemap(df_table.head(30), path=['all', "items"], values='incident_count',
                  color=df_table["incident_count"].head(30), hover_data=['items'],
                  color_continuous_scale='Blues',
                  )
fig.show()

### Dataframe splitting

Creating two dataframes:
1. Orders: will contain invoice and stockcodes of items bought in that invoice
2. Products: Mapping between stockcode and its description

In [ ]:
# DataFrame for building the recommendation system
orders = df2[['InvoiceNo', 'StockCode']].copy()
# DataFrame for retrieving product descriptions
products = df2[['StockCode', 'Description']].copy()

In [ ]:
orders= orders.groupby('InvoiceNo')['StockCode'].apply(list).reset_index(name="cart_items")

In [ ]:
orders

,InvoiceNo,cart_items
0,536365,"[_85123A, _71053, _84406B, _84029G, _84029E, _..."
1,536366,"[_22633, _22632]"
2,536367,"[_84879, _22745, _22748, _22749, _22310, _8496..."
3,536368,"[_22960, _22913, _22912, _22914]"
4,536369,[_21756]
...,...,...
23462,C581470,[_23084]
23463,C581484,[_23843]
23464,C581490,"[_22178, _23144]"
23465,C581568,[_21258]


Some  items had multiple description for a given stockcode, so only keeping the top description for each stockcode

In [ ]:
# Keep only the first Description of each product
products = products[~products.duplicated(subset=['StockCode'])]

In [ ]:
# Set the index to StockCode for easy lookups
products = products.set_index('StockCode')
products = products['Description']

### Recommendation System

1. Convert the orders dataframe into **transaction encoding,** that is for every invoice it will give if a stockcode is present or not as true/false.</br>
It is much similar to one hot encoding

In [ ]:
te = TransactionEncoder()
# fit the TransactionEncoder
# do the transformation
orders_1hot = te.fit(orders['cart_items']).transform(orders['cart_items'])
# orders_1hot
# te.columns_

In [ ]:
# convert orders_1hot to a DataFrame
orders_1hot = pd.DataFrame(orders_1hot, columns = te.columns_)
# Inspect the first 5 rows of the DataFrame
orders_1hot

,_10002,_10080,_10120,_10123C,_10124A,_10124G,_10125,_10133,_10135,_11001,...,_DCGS0004,_DCGS0069,_DCGS0070,_DCGS0076,_DCGSSBOY,_DCGSSGIRL,_DOT,_PADS,_POST,_S
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23462,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
23463,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
23464,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
23465,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
dataset = orders_1hot

In [ ]:
dataset

,_10002,_10080,_10120,_10123C,_10124A,_10124G,_10125,_10133,_10135,_11001,...,_DCGS0004,_DCGS0069,_DCGS0070,_DCGS0076,_DCGSSBOY,_DCGSSGIRL,_DOT,_PADS,_POST,_S
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23462,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
23463,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
23464,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
23465,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
# Convert dataset into 1-0 encoding from True/False

def encode_units(x):
    if x == False:
        return 0 
    if x == True:
        return 1
    
dataset = dataset.applymap(encode_units)
dataset.head(10)

,_10002,_10080,_10120,_10123C,_10124A,_10124G,_10125,_10133,_10135,_11001,...,_DCGS0004,_DCGS0069,_DCGS0070,_DCGS0076,_DCGSSBOY,_DCGSSGIRL,_DOT,_PADS,_POST,_S
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


2. **Using apriori from mlxtend to create frequent itemsets, i.e. all the itemsets that have support >= 0.01**

**Itemsets:** Items that are being bought together in one transcation

By **support,** we are calculating how frequent is that particular in all the transactions.

To calculate **support,** (Count of occurence of that itemset in all transaction)/(Total no. oftransactions)

In [ ]:
frequent_itemsets = apriori(dataset, min_support = 0.01, use_colnames=True)

In [ ]:
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))

In [ ]:
frequent_itemsets

,support,itemsets,length
0,0.022244,(_15036),1
1,0.016534,(_15056BL),1
2,0.023309,(_15056N),1
3,0.010611,(_16161P),1
4,0.012400,(_16237),1
...,...,...,...
1365,0.010355,"(_20724, _20719, _20723, _22356)",4
1366,0.010738,"(_20724, _20719, _22355, _22356)",4
1367,0.010525,"(_22356, _20724, _22355, _20723)",4
1368,0.011463,"(_22386, _85099B, _21931, _22411)",4


3. Creating **association rules** from these itmesets, using association_rules from mlxtend library

We consider only those rules where lift is >= 1.2

**Antecedents:** Items that we are buying initially</br>
**Consequets:** Items that we are buying in response to antecedent


**Lift:** Gives the likelihood that if item A is being bought then how likely is item B is to be bought given how popular B is.</br>
If lift(A->B) > 1, it is likely that item B is bought if item A is bought and vice versa


In [ ]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.2)
rules["antecedents_length"] = rules["antecedents"].apply(lambda x: len(x))
rules["consequents_length"] = rules["consequents"].apply(lambda x: len(x))
rules.sort_values("lift",ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedents_length,consequents_length
1829,"(_23170, _23171)",(_23172),0.013721,0.013380,0.010994,0.801242,59.881374,0.010811,4.963929,2,1
1832,(_23172),"(_23170, _23171)",0.013380,0.013721,0.010994,0.821656,59.881374,0.010811,5.530205,1,2
1833,(_23171),"(_23170, _23172)",0.016449,0.011591,0.010994,0.668394,57.666165,0.010804,2.980672,1,2
1828,"(_23170, _23172)",(_23171),0.011591,0.016449,0.010994,0.948529,57.666165,0.010804,19.108998,2,1
961,(_23171),(_23172),0.016449,0.013380,0.012017,0.730570,54.599634,0.011797,3.661876,1,1
...,...,...,...,...,...,...,...,...,...,...,...
174,(_20725),(_22423),0.068522,0.092428,0.011846,0.172886,1.870496,0.005513,1.097275,1,1
753,(_22423),(_85123A),0.092428,0.098521,0.015383,0.166436,1.689341,0.006277,1.081475,1,1
752,(_85123A),(_22423),0.098521,0.092428,0.015383,0.156142,1.689341,0.006277,1.075503,1,1
750,(_85099B),(_22423),0.090979,0.092428,0.012273,0.134895,1.459461,0.003864,1.049089,1,1


4. Doing prediction using the antecedents and consequents

Look up the given item stockcode in the antecedents and iterate over all the consequents.</br>
These consequents are the items to be recommended.

In [ ]:
def predict(antecedent, rules, max_results= 6):
    
    # get the rules for this antecedent
    preds = rules[rules['antecedents'] == antecedent]
    
    # a way to convert a frozen set with one element to string
    preds = preds['consequents'].apply(iter).apply(next)
    
    return preds[:max_results]

In [ ]:
preds = predict({'_85123A'}, rules)
preds

201    _20725
264    _20727
290    _20728
293    _20914
369    _21212
377    _21232
Name: consequents, dtype: object

In [ ]:
print(products['_85123A'])

WHITE HANGING HEART T-LIGHT HOLDER


In [ ]:
for stockid in preds:  
    print(products[stockid])

LUNCH BAG RED RETROSPOT
LUNCH BAG  BLACK SKULL.
LUNCH BAG CARS BLUE
SET/5 RED RETROSPOT LID GLASS BOWLS
PACK OF 72 RETROSPOT CAKE CASES
STRAWBERRY CERAMIC TRINKET BOX
